### For exploring the difference between manual compression/decompressiong vs command line decoding and encoding. 

In [ ]:
DEVICE='cpu' # either 'cuda' or 'cpu'

%pwd
%cd /app

import os
import dac
from audiotools import AudioSignal
import torch

import numpy as np
import matplotlib.pyplot as plt
import soundfile as sf
import IPython.display as ipd

import math
import matplotlib.pyplot as plt
import numpy as np

In [ ]:
if DEVICE=='cuda' :
    if torch.cuda.is_available() :
        nd=torch.cuda.device_count()
        for i in range(nd) : 
            torch.cuda.get_device_properties(i).total_memory/1e9
            print(f'memeory on cuda {i} is  {torch.cuda.get_device_properties(i).total_memory/1e9}')
        device = torch.device('cuda') # if the docker was started with --gpus all, then can choose here with cuda:0 (or cpu)
    else :
        device = torch.device('cuda')

device = torch.device(DEVICE)
print(f'Device is {device}')


In [ ]:
model_path = dac.utils.download(model_type="44khz") 
model = dac.DAC.load(model_path)

model.to(device); #wanna see the model? remove the semicolon
model.eval();  # need to be "in eval mode" in order to set the number of quantizers

In [ ]:
dataroot="dacdevdata" 
!ls ./{dataroot}

In [ ]:
N_QUANTIZERS=4

signal_filename = dataroot+'/44kHz/N4/PisWinAppBee_sparse_recon/DSApplause--numClappers_exp-00.50.wav'
signal_dacfilelame = dataroot+'/44kHz/N4/PisWinAppBee_sparse_dac/DSApplause--numClappers_exp-00.50.dac'

with torch.no_grad():
    signal = AudioSignal(signal_filename)
    signal_compressed = model.compress(signal, n_quantizers=N_QUANTIZERS)  # RET: dac_file
    signal_compressed.save(signal_dacfilelame)

    # for testing and comparison:
    signal_audio = signal.cpu().detach().numpy()[0,0,:]

    signal.to(model.device)
    signal_prep = model.preprocess(signal.audio_data, signal.sample_rate)
    with torch.no_grad():
        signal_encoded_z, signal_encoded_codes, signal_encoded_latents, _, _ = model.encode(signal_prep, N_QUANTIZERS) # model.encode(snd2_x, 4)

    print(f'--')
    print(f'signal_compressed.codes shape is: {signal_compressed.codes.shape}')
    print(f'signal_encoded_codes shape is: {signal_encoded_codes.shape}') 
    print(f'--')
#----------------------------------------
    signal_dacfile = dac.DACFile.load(signal_dacfilelame)
    print(f'dacfile.codes shape is: {signal_dacfile.codes.shape}')

    print(f'--')
    # Is signal_compressed the same as signal_dacfile?
    print(f' signal_compressed.codes == signal_dacfile.codes? ..... {signal_compressed.codes == signal_dacfile.codes}')
    print(f' signal_compressed.chunk_length == signal_dacfile.chunk_length? ..... {signal_compressed.chunk_length == signal_dacfile.chunk_length}')
    print(f' signal_compressed.sample_rate == signal_dacfile.sample_rate? ..... {signal_compressed.sample_rate == signal_dacfile.sample_rate}')
    print(f' signal_compressed.padding == signal_dacfile.padding? ..... {signal_compressed.padding == signal_dacfile.padding}')
    print(f' signal_compressed.channels == signal_dacfile.channels? ..... {signal_compressed.channels == signal_dacfile.channels}')
    print(f' signal_compressed.original_length == signal_dacfile.original_length? ..... {signal_compressed.original_length == signal_dacfile.original_length}')
    print(f'------------------')
#-----------------------------------------
    #First decompress dacfile
    signal_decompressed = model.decompress(signal_dacfile, verbose=False)  #an AudioSignal
    audio_decompressed = signal_decompressed.cpu().detach().numpy()[0,0,:]

    #Now decode dacfile
    z_from_c, l_from_c,c =model.quantizer.from_codes(signal_dacfile.codes.to(device))
    print(f'z_from_c shape is: {z_from_c.shape}')
    print(f'l_from_c shape is: {l_from_c.shape}')
    print(f'c shape is: {c.shape}')

    zq, zp, c =model.quantizer.from_latents(l_from_c)
    print(f'z_from_c is {z_from_c}')
    print(f'zq is {zq}')
    signal_decode_tensor = model.decode(zq) ####  (z_from_c) ### 

    
    audio_decoded = signal_decode_tensor.cpu().detach().numpy()[0,0,:]

    print(f'------------------')
    
#-------------------------------
    print(f'ORIGINAL AudioSignal has length = {str(len(signal_audio))}')
    print(f'Mannually decoded AudioSignal has length = {str(len(audio_decoded))}')
    print(f'DECOMPRESSED AudioSignal has length = {str(len(audio_decompressed))}')
    

    print(f'------------------')
    print(f' --------  compare to embeddings from original signal    --------')
    print(f' signal_encoded_z shape is: { signal_encoded_z.shape}')
    print(f'signal_encoded_latents shape is: {signal_encoded_latents.shape}')
    print(f'signal_encoded_codes shape is: {signal_encoded_codes.shape}')   


In [ ]:
#test
# load the file, and then 
with torch.no_grad():
    dacfile = dac.DACFile.load(dataroot+'/44kHz/N4/PisWinAppBee_sparse_dac/DSApplause--numClappers_exp-00.50.dac')
    
    # FIRST - Decompress it back to an AudioSignal\ from codes to z (1024) to signal   
    print(f'dacfile.codes shape is: {dacfile.codes.shape}')
    z_from_c, l_from_c,c =model.quantizer.from_codes(dacfile.codes.to(device))
    print(f'z_from_c shape is: {z_from_c.shape}')
    print(f'l_from_c shape is: {l_from_c.shape}')
    print(f'c shape is: {c.shape}')
    xTensor = model.decode(z_from_c)

    # SECOND - Decompress directly using model.decompress   
    yAudioSig = model.decompress(dacfile)  #an AudioSignal

    
print(f' The length of  xTensor  signal is {len(xTensor.cpu().detach().numpy()[0,0,:])}')
print(f' The length of yAudioSig signal is {len(yAudioSig.cpu().detach().numpy()[0,0,:])}')

In [ ]:
xsig=xTensor.cpu().detach().numpy()[0,0,:]
plt.plot(xsig)
ipd.Audio(xsig, rate=44100)

In [ ]:


ysig=yAudioSig.cpu().detach().numpy()[0,0,:]
plt.plot(ysig)
ipd.Audio(ysig, rate=44100)


In [ ]:
len(xsig)

In [ ]:
len(ysig)

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

def plot_arrays(arr1, arr2):
    """
    Takes two arrays, pads the shorter one with zeros at the end to make them of equal length,
    and plots both on a Matplotlib graph with different colors.

    Args:
    arr1 (np.array): First input array.
    arr2 (np.array): Second input array.
    """
    # Determine the maximum length of the two arrays
    max_length = max(len(arr1), len(arr2))

    # Pad arrays to the maximum length
    padded_arr1 = np.pad(arr1, (0, max_length - len(arr1)), mode='constant')
    padded_arr2 = np.pad(arr2, (0, max_length - len(arr2)), mode='constant')

    # Create the plot
    plt.figure(figsize=(10, 5))
    plt.plot(padded_arr1, label='Array 1', color='blue')
    plt.plot(padded_arr2, label='Array 2', color='green')
    plt.title('Comparison of Two Arrays')
    plt.xlabel('Index')
    plt.ylabel('Value')
    plt.legend()
    plt.grid(True)
    plt.show()



In [ ]:
plot_arrays(xsig[8000+4096:(25000+4096)], ysig[8000:35000])

In [ ]:
plot_arrays(xsig[25000+4096:(50000+4096)], ysig[25000:50000])

In [ ]:
plot_arrays(xsig, ysig)